# **Zillow Clustering Project**

## Project Goals
- Create documented files to clean and prepare Zillow dataset for processing by clustering algorithm for further modeling via a regression ML algorithm.
- Use clustering to identify driving features of log error and to potentially create new feature combinations or simply to use the cluster as a driving feature.
- Evaluate the clusters and try to extrapolate usefull info from them.
- Pass updated dataframe with new features/chosen features/cluster features to a regression model.
- Present on my final Jupyter Notebook, giving a high-level overview of the process used to create the clusters and how the information garnered from them is useful to my regression model.

## Initial Hypotheses:
***
>   - $H_{i}$: The key drivers of log error will be linked to tax value clusters and area clusters
***

## Plan:
- [x] Create repo on github to save all files related to the project.
- [x] Create README.md with goals, initial hypotheses, data dictionary, and outline plans for the project in a trello board.
- [x] Acqiure zillow data using acquire.py file drawing directly from Codeups `zillow` database with SQL queries. Create functions for use in conjunction with prepare.py.
- [] Clean, tidy, and encode data in such a way that it is usable in a machine learning algorithm. Includes dropping unneccesary columns, creating dummies where needed and changing string values to numeric values and getting rid of outliers
- [] Utilize recursive feature elimination and clustering algorithms to search the data for meaningful driving features of log error.
- [] Create hypotheses based on preliminary statistical tests
- [] Test hypotheses with tests such as t-test, chi-squared to determine the viability of said hypotheses by comparing p-values to alpha.
- [] Establish a baseline accuracy.
- [] Train three different classification models from OLS, GLM, and Lasso + Lars, testing a variety of parameters and features, both engineered and pre-existing.
- [] Evaluate models using RMSE, R^2 score, and other metrics on in-sample and out-of-sample datasets.
- [] Once a single, best preforming model has been chosen, evaluate the preformance of the model on the test dataset.
- [] Present my jupyter notebook to Codeup instructors

In [1]:
# Imports from my .py scripts and modules necessary 
# for modeling and exploring data 
import wrangle as wr
import explore as ex


import random
from sklearn.preprocessing import PolynomialFeatures
import sklearn.preprocessing
import scipy.stats as stats
from sklearn.cluster import KMeans

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import itertools
from datetime import date

import warnings
warnings.filterwarnings("ignore")

## Wrangle

### Acquire

In [2]:
# Acquire Zillow data from MySQL database

zillow_raw = wr.acquire_zillow()

In [3]:
zillow_raw.shape

(77381, 67)

In [4]:
# Sample of raw data
zillow_raw.head().T

id,1727539,1387261,11677,2288172,1970746
parcelid,14297519,17052889,14186244,12177905,10887214
airconditioningtypeid,NaN,NaN,NaN,NaN,1.0
architecturalstyletypeid,NaN,NaN,NaN,NaN,NaN
basementsqft,NaN,NaN,NaN,NaN,NaN
bathroomcnt,3.5,1.0,2.0,3.0,3.0
...,...,...,...,...,...
buildingclassdesc,None,None,None,None,None
heatingorsystemdesc,None,None,None,Central,Central
propertylandusedesc,Single Family Residential,Single Family Residential,Single Family Residential,Single Family Residential,Condominium
storydesc,None,None,None,None,None


In [5]:
# Overview of raw data
wr.summarize(zillow_raw)



Dataframe info: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 77381 entries, 1727539 to 2455685
Data columns (total 67 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77381 non-null  int64  
 1   airconditioningtypeid         24953 non-null  float64
 2   architecturalstyletypeid      206 non-null    float64
 3   basementsqft                  50 non-null     float64
 4   bathroomcnt                   77381 non-null  float64
 5   bedroomcnt                    77381 non-null  float64
 6   buildingclasstypeid           15 non-null     float64
 7   buildingqualitytypeid         49672 non-null  float64
 8   calculatedbathnbr             76772 non-null  float64
 9   decktypeid                    614 non-null    float64
 10  finishedfloor1squarefeet      6023 non-null   float64
 11  calculatedfinishedsquarefeet  77185 non-null  float64
 12  finishedsquarefeet12          737

(0.9989, 0.9992]        0
(0.9992, 0.9994]        0
(0.9994, 0.9996]        0
(0.9996, 0.9998]        0
(0.9998, 1.0]       15042
(1.0, 1.0002]           0
(1.0002, 1.0004]        0
(1.0004, 1.0006]        0
(1.0006, 1.0008]        0
(1.0008, 1.001]         0
Name: pooltypeid7, dtype: int64
0100    26714
122     13045
010C     8813
0101     6399
34       4922
        ...  
0115        1
0131        1
1432        1
040G        1
020E        1
Name: propertycountylandusecode, Length: 75, dtype: int64
(30.755, 55.4]       15
(55.4, 79.8]          0
(79.8, 104.2]         0
(104.2, 128.6]        0
(128.6, 153.0]        0
(153.0, 177.4]        0
(177.4, 201.8]        0
(201.8, 226.2]        0
(226.2, 250.6]     3271
(250.6, 275.0]    74095
Name: propertylandusetypeid, dtype: int64
LAR1         6747
LAR3         2437
LARS         1389
LBR1N        1222
LARD1.5      1127
             ... 
PSR47200*       1
LBRR2I          1
PD SP           1
AVR2*           1
BFA15000*       1
Name: propertyzo

                          num_rows_missing  percent_rows_missing
parcelid                                 0              0.000000
airconditioningtypeid                52428             67.753066
architecturalstyletypeid             77175             99.733785
basementsqft                         77331             99.935385
bathroomcnt                              0              0.000000
...                                    ...                   ...
buildingclassdesc                    77366             99.980615
heatingorsystemdesc                  27941             36.108347
propertylandusedesc                      0              0.000000
storydesc                            77331             99.935385
typeconstructiondesc                 77159             99.713108

[67 rows x 2 columns]
nulls in dataframe by row: 
   num_cols_missing percent_cols_missing     id
0                23    34.32835820895522      2
1                24    35.82089552238806     13
2                25     3

### Prepare

In [6]:
# Prepping raw zillow data, imputing in year_built,
# dropping nulls, outliers and columns with too many null vals
# such as storytypeid with only 50 non-null vals

zillow_prep = wr.wrangle_zillow()

In [7]:
zillow_prep.shape

(50864, 25)

In [16]:
# Dropped column/row count
print(f'Dropped {zillow_raw.shape[1] - zillow_prep.shape[1]} \
columns. \nDropped {zillow_raw.shape[0] - zillow_prep.shape[0]} rows.')

Dropped 42 columns. 
Dropped 26517 rows.


In [9]:
zillow_prep.head().T

id,1727539,1387261,11677,2288172,781532
bathrooms,3.5,1.0,2.0,3.0,3.0
bedrooms,4.0,2.0,3.0,4.0,4.0
quality_id,6.0,6.0,6.0,8.0,9.0
home_area,3100.0,1465.0,1243.0,2376.0,2962.0
latitude,33634931.0,34449266.0,33886168.0,34245180.0,34145202.0
longitude,-117869207.0,-119281531.0,-117823170.0,-118240722.0,-118179824.0
lot_area,4506.0,12647.0,8432.0,13038.0,63000.0
census_tb,60590630.072012,61110010.023006,60590218.022012,60373001.001006,60374608.001014
city_id,53571.0,13091.0,21412.0,396551.0,47019.0
county_id,1286.0,2061.0,1286.0,3101.0,3101.0


In [10]:
wr.summarize(zillow_prep)



Dataframe info: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50864 entries, 1727539 to 2455685
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   bathrooms       50864 non-null  float64
 1   bedrooms        50864 non-null  float64
 2   quality_id      50864 non-null  float64
 3   home_area       50864 non-null  float64
 4   latitude        50864 non-null  float64
 5   longitude       50864 non-null  float64
 6   lot_area        50864 non-null  float64
 7   census_tb       50864 non-null  float64
 8   city_id         50864 non-null  float64
 9   county_id       50864 non-null  float64
 10  zip_code        50864 non-null  float64
 11  roomcnt         50864 non-null  float64
 12  unitcnt         50864 non-null  float64
 13  structure_tax   50864 non-null  float64
 14  tax_value       50864 non-null  float64
 15  year_assessed   50864 non-null  float64
 16  land_tax_value  50864 non-null  float64
 17  year

(0.9989, 0.9992]        0
(0.9992, 0.9994]        0
(0.9994, 0.9996]        0
(0.9996, 0.9998]        0
(0.9998, 1.0]       50864
(1.0, 1.0002]           0
(1.0002, 1.0004]        0
(1.0004, 1.0006]        0
(1.0006, 1.0008]        0
(1.0008, 1.001]         0
Name: unitcnt, dtype: int64
(-2949.525, 309900.4]     43692
(309900.4, 619652.8]       5428
(619652.8, 929405.2]       1082
(929405.2, 1239157.6]       415
(1239157.6, 1548910.0]      147
(1548910.0, 1858662.4]       57
(1858662.4, 2168414.8]       25
(2168414.8, 2478167.2]       12
(2478167.2, 2787919.6]        2
(2787919.6, 3097672.0]        4
Name: structure_tax, dtype: int64
(-1735.879, 502241.8]     33695
(502241.8, 1001229.6]     12222
(1001229.6, 1500217.4]     2727
(1500217.4, 1999205.2]     1112
(1999205.2, 2498193.0]      476
(2498193.0, 2997180.8]      293
(2997180.8, 3496168.6]      153
(3496168.6, 3995156.4]       91
(3995156.4, 4494144.2]       61
(4494144.2, 4993132.0]       34
Name: tax_value, dtype: int64
(2013.98